In [249]:
import pandas as pd
import numpy as np
import glob


df = pd.concat((pd.read_csv(f, delimiter='|', header=None, \
names=['feccandid','candname','party','date', 'state', 'chamber',\
'district','incumb.challeng', 'cand_status', '1', '2','3','4', '5', '6'],\
usecols=['feccandid', 'party', 'date', 'state', 'chamber'])for f in \
glob.glob('/home/jayaramdas/anaconda3/Thesis/FEC/cn_data/cn**.txt')))


df = df[(df['date'] > 2005) & (df['date'] < 2015)]

(df).head(1)

df.to_csv('Thesis/FEC/can_master_cln', index=False)

In [250]:
year_bill_amount=pd.read_csv('year_bill_amount.sum')

In [251]:
year_bill_amount.rename(columns={'id.thomas':'thomasid'}, inplace=True)

In [252]:
year_bill_amount.rename(columns={'feccanid':'feccandid'}, inplace=True)

In [253]:
year_bill_amount.head(1)

,cid,date,unemployment,catcode,billsum,thomasid,loansum,feccandid,amtsum
0,N00013870,2007.0,4.6,H0000,1.0,1697,0.0,S2MN00126,1.0


In [254]:
year_bill_amount.describe()

,date,unemployment,billsum,thomasid,loansum,amtsum
count,454384.000000,454384.000000,454384.000000,454384.000000,454384.000000,454384.000000
mean,2009.746547,0.126262,0.020214,1434.189368,0.136428,6.738634
std,2.812095,0.987317,0.165285,600.620948,0.906749,62.011896
min,2005.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,2007.000000,0.000000,0.000000,1010.000000,0.000000,1.000000
50%,2010.000000,0.000000,0.000000,1651.000000,0.000000,2.500000
75%,2012.000000,0.000000,0.000000,1869.000000,0.000000,5.500000
max,2014.000000,13.700000,6.000000,2291.000000,207.732000,19739.002000


In [255]:
year_bill_amount['date_fwd1']=year_bill_amount.date-1

In [256]:
fwdamtsum=year_bill_amount[['cid', 'date_fwd1', 'amtsum', 'catcode']].copy()
fwdamtsum.rename(columns={'date_fwd1':'date', 'amtsum' : 'amtsumfwd_1'},\
                 inplace=True)
year_bill_amount.drop('date_fwd1', axis=1, inplace=True)

In [257]:
year_bill_amount=pd.merge(fwdamtsum,year_bill_amount,on=['date', 'cid', 'catcode'], how=('right'))
year_bill_amount.fillna(value=0, inplace=True)
#year_bill_amount.drop('feccandid', axis=1, inplace=True)

In [258]:
year_bill_amount['date_lag1']=year_bill_amount.date+1

In [259]:
lagamtsum=year_bill_amount[['cid', 'date_lag1', 'amtsum', 'catcode']].copy()
lagamtsum.rename(columns={'date_lag1':'date', 'amtsum' : 'amtsumlag_1'},\
                 inplace=True)
year_bill_amount.drop('date_lag1', axis=1, inplace=True)

In [260]:
year_bill_amount=pd.merge(lagamtsum,year_bill_amount,on=['date', 'cid', 'catcode'], how=('right'))
year_bill_amount.fillna(value=0, inplace=True)
#+year_bill_amount.drop('feccandid', axis=1, inplace=True)

In [261]:
year_bill_amount.describe()

,date,amtsumlag_1,amtsumfwd_1,unemployment,billsum,thomasid,loansum,amtsum
count,746730.000000,746730.000000,746730.000000,746730.000000,746730.000000,746730.000000,746730.000000,746730.000000
mean,2009.919962,5.872943,6.148951,0.159265,0.025079,1446.632614,0.161361,8.035435
std,2.677255,97.705132,47.312040,1.113109,0.183716,593.250792,0.920925,68.808872
min,2005.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,2008.000000,0.000000,0.000000,0.000000,0.000000,1151.000000,0.000000,1.000000
50%,2010.000000,2.000000,2.000000,0.000000,0.000000,1647.000000,0.000000,3.000000
75%,2012.000000,5.000000,5.000000,0.000000,0.000000,1866.000000,0.000000,6.500000
max,2014.000000,19739.002000,10106.424000,13.700000,6.000000,2291.000000,207.732000,19739.002000


In [262]:
len(year_bill_amount)#.head(1)

746730

In [263]:
year_bill_amount.cid.nunique()

915

In [264]:
year_bill_amount.isnull().sum()

cid             0
date            0
amtsumlag_1     0
catcode         0
amtsumfwd_1     0
unemployment    0
billsum         0
thomasid        0
loansum         0
feccandid       0
amtsum          0
dtype: int64

In [265]:
idindex=pd.read_csv('fec_thom_cid_index')
idindex.rename(columns={'id.thomas':'thomasid'},\
          inplace=True)

In [266]:
tz = pd.read_csv('histcurrent.merged.csv',low_memory=False,\
                 usecols=['id.thomas', 'type', 'party', 'state', 'end'])
tz.drop_duplicates('id.thomas', keep='first', inplace=True)
tz.rename(columns={'id.thomas':'thomasid'},\
          inplace=True)
tz['end'] = \
tz['end'].map(lambda x: str(x)[:-6])
tz.rename(columns={'end':'date'}, inplace=True)

tz=pd.merge(tz,idindex, how='left',\
                          on=['thomasid'])
tz.drop(['thomasid', 'cid'], axis=1, inplace=True)
#year_bill_amount.fillna(value=0, inplace=True)
tz.rename(columns={'type':'chamber'}, inplace=True)
tz=tz[['feccandid', 'party', 'date', 'state', 'chamber'] ]
tz.head(1)

,feccandid,party,date,state,chamber
0,S4TN00153,Republican,2001,TN,sen


In [267]:
tz.feccandid.nunique()

981

In [268]:
tz['date'] = tz['date'].astype('float64')

In [269]:
tz.isnull().sum()

feccandid    3
party        0
date         0
state        0
chamber      0
dtype: int64

In [270]:
canmaster=pd.read_csv('Thesis/FEC/can_master_cln')
canmaster.drop_duplicates(['feccandid', 'date'], inplace=True)

In [271]:
canmaster.loc[canmaster.chamber == 'H', ['chamber']] = 'rep'

In [272]:
canmaster.loc[canmaster.chamber == 'S', ['chamber']] = 'sen'

In [273]:
canmaster=canmaster[(canmaster['chamber'] != 'P')]

In [274]:
canmaster.columns.tolist()

['feccandid', 'party', 'date', 'state', 'chamber']

In [275]:
canmaster=canmaster[(canmaster['party']=='REP')|(canmaster['party']=='DEM')\
                   |(canmaster['party']=='IND')]
#dz = df[(df['date'] > '01/03/2005') & (df['date'] < '01/03/2015')]

In [276]:
canmaster.loc[canmaster.party == 'REP', ['party']] = 'Republican'

In [277]:
canmaster.loc[canmaster.party == 'DEM', ['party']] = 'Democrat'

In [278]:
canmaster.loc[canmaster.party == 'IND', ['party']] = 'Independent'

In [279]:
canmaster.party.nunique()

3

In [280]:
canmaster.head(1)

,feccandid,party,date,state,chamber
0,H0AK00097,Republican,2014,AK,rep


In [281]:
len (canmaster)

11776

In [282]:
frames=[canmaster, tz]
canmaster=pd.concat(frames)

In [283]:
canmaster.feccandid.nunique()

8682

In [284]:
canmaster.drop_duplicates(['feccandid', 'date'], inplace=True)

In [285]:
#canmaster=pd.merge(canmaster,idindex, how='left',\
    #                      on=['feccandid'])
#year_bill_amount.fillna(value=0, inplace=True)

In [286]:
year_bill_amount = year_bill_amount.drop_duplicates(keep='first')

In [287]:
canmaster = canmaster.drop_duplicates(keep='first')

In [288]:
yba=pd.merge(canmaster,year_bill_amount, how='right',\
                          on=['feccandid', 'date'])


In [289]:
yba = yba.drop_duplicates(keep='first')

In [290]:
yba.isnull().sum()

feccandid            0
party           182200
date                 0
state           182200
chamber         182200
cid                  0
amtsumlag_1          0
catcode              0
amtsumfwd_1          0
unemployment         0
billsum              0
thomasid             0
loansum              0
amtsum               0
dtype: int64

In [291]:
yba.describe()

,date,amtsumlag_1,amtsumfwd_1,unemployment,billsum,thomasid,loansum,amtsum
count,412793.000000,412793.000000,412793.000000,412793.000000,412793.000000,412793.000000,412793.000000,412793.000000
mean,2009.708941,3.840257,4.159154,0.122742,0.019717,1433.625495,0.134406,6.443382
std,2.816359,38.048633,31.242003,0.972069,0.163290,600.111597,0.904576,53.010064
min,2005.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,2007.000000,0.000000,0.000000,0.000000,0.000000,1003.000000,0.000000,1.000000
50%,2010.000000,1.000000,1.000000,0.000000,0.000000,1655.000000,0.000000,2.500000
75%,2012.000000,4.000000,4.050000,0.000000,0.000000,1871.000000,0.000000,5.500000
max,2014.000000,19739.002000,10106.424000,13.700000,6.000000,2291.000000,207.732000,19739.002000


In [292]:
df=pd.read_csv('billspon.legisl.merg.3', usecols=['date', 'id.thomas',\
                                                'leg_totalbills'])
#df.drop_duplicates(['date', 'id.thomas'],inplace=True)
df.rename(columns={'id.thomas':'thomasid'}, inplace=True)
df.head(1)

,date,thomasid,leg_totalbills
0,2006.0,1440,14


In [293]:
df.isnull().sum()

date              0
thomasid          0
leg_totalbills    0
dtype: int64

In [294]:
df.thomasid.nunique()

844

In [295]:
df = df.drop_duplicates(keep='first')

In [296]:
yba.head(1)

,feccandid,party,date,state,chamber,cid,amtsumlag_1,catcode,amtsumfwd_1,unemployment,billsum,thomasid,loansum,amtsum
0,H0AL02087,Republican,2014.0,AL,rep,N00030768,1.0,E1110,0.0,0.0,0.0,1986,0.0,2.0


In [297]:
yba2=pd.merge(df,yba, how='right',\
                          on=['thomasid', 'date'])


In [298]:
yba2.describe()

,date,thomasid,leg_totalbills,amtsumlag_1,amtsumfwd_1,unemployment,billsum,loansum,amtsum
count,412793.000000,412793.000000,365699.000000,412793.000000,412793.000000,412793.000000,412793.000000,412793.000000,412793.000000
mean,2009.708941,1433.625495,9.965349,3.840257,4.159154,0.122742,0.019717,0.134406,6.443382
std,2.816359,600.111597,11.100884,38.048633,31.242003,0.972069,0.163290,0.904576,53.010064
min,2005.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2007.000000,1003.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2010.000000,1655.000000,7.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.500000
75%,2012.000000,1871.000000,12.000000,4.000000,4.050000,0.000000,0.000000,0.000000,5.500000
max,2014.000000,2291.000000,156.000000,19739.002000,10106.424000,13.700000,6.000000,207.732000,19739.002000


In [299]:
yba2.isnull().sum()

date                   0
thomasid               0
leg_totalbills     47094
feccandid              0
party             182200
state             182200
chamber           182200
cid                    0
amtsumlag_1            0
catcode                0
amtsumfwd_1            0
unemployment           0
billsum                0
loansum                0
amtsum                 0
dtype: int64

In [300]:
yba2.cid.nunique()

915

In [301]:
yba2.drop_duplicates(['cid', 'date'], inplace=True)

In [302]:
crsp = pd.read_csv('crsp.dime.todf') 

In [303]:
crsp.drop_duplicates(['date', 'cid'], inplace=True, keep='first')

In [304]:
yba3= pd.merge(crsp,yba2, on=['date', \
                'cid'], how='right')

In [305]:
yba3.count()

date               6459
cid                6459
stdv_paccfscore    6129
cfcontrol           443
thomasid           6459
leg_totalbills     5090
feccandid          6459
party              3132
state              3132
chamber            3132
amtsumlag_1        6459
catcode            6459
amtsumfwd_1        6459
unemployment       6459
billsum            6459
loansum            6459
amtsum             6459
dtype: int64

In [306]:
yba3.head()

,date,cid,stdv_paccfscore,cfcontrol,thomasid,leg_totalbills,feccandid,party,state,chamber,amtsumlag_1,catcode,amtsumfwd_1,unemployment,billsum,loansum,amtsum
0,2006.0,N00000528,0.424150,0.921787,1010.0,5.0,H8VT01016,Independent,VT,rep,1.0,H1710,0.0,0.0,0.0,0.0,7.000
1,2006.0,N00027860,0.360972,0.761430,2123.0,NaN,H6IL06141,Democrat,IL,rep,3.0,Z1200,0.0,0.0,0.0,0.0,105.668
2,2006.0,N00028138,0.254904,0.831541,2173.0,NaN,H6HI02244,Democrat,HI,rep,0.0,LG000,0.0,0.0,0.0,0.0,0.500
3,2006.0,N00027509,0.214621,-1.011639,1997.0,NaN,H6CO03139,Republican,CO,rep,0.0,J2200,0.0,0.0,0.0,0.0,77.000
4,2006.0,N00028058,0.435242,0.629428,1822.0,NaN,S6VA00127,Democrat,VA,sen,0.0,K1200,0.5,0.0,0.0,0.0,1.000


In [307]:
yba3.describe()

,date,stdv_paccfscore,cfcontrol,thomasid,leg_totalbills,amtsumlag_1,amtsumfwd_1,unemployment,billsum,loansum,amtsum
count,6459.000000,6129.000000,443.000000,6459.000000,5090.000000,6459.000000,6459.000000,6459.000000,6459.000000,6459.000000,6459.000000
mean,2009.718223,0.362302,-0.123252,1445.171079,9.871906,10.854209,10.819183,1.352516,0.216752,0.402772,14.477967
std,2.812758,0.095944,0.776772,604.344327,10.466637,81.978453,39.535877,2.948311,0.497212,1.354806,54.474334
min,2005.000000,0.000000,-1.872674,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2007.000000,0.297496,-0.841893,1010.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,2010.000000,0.360520,-0.332453,1667.000000,7.000000,3.000000,3.000000,0.000000,0.000000,0.000000,5.000000
75%,2012.000000,0.421292,0.633367,1892.000000,12.000000,8.500000,9.500000,0.000000,0.000000,0.000000,11.500000
max,2014.000000,1.381687,1.302073,2291.000000,156.000000,5273.379000,1526.488000,13.700000,6.000000,44.706000,1693.960000


In [308]:
rr=pd.read_csv('Thesis/leg_years.csv', usecols=['years', 'id.thomas', \
                                               'served'])


In [309]:
rr.rename(columns={'years':'date'}, inplace=True)


In [310]:
rr.rename(columns={'id.thomas':'thomasid'}, inplace=True)


In [311]:
rr['years_exp']=rr.groupby(['thomasid']).served.cumsum()


In [312]:
rr.drop(['served'], axis=1, inplace=True)


In [313]:
rr = rr[(rr['date'] > 2004)]

In [314]:
year_bill_amount= pd.merge(rr,yba3, on=['date', \
                'thomasid'], how='right')

In [315]:
rr.head(1)

,date,thomasid,years_exp
55,2005,2,17


In [316]:
yba3.isnull().sum()

date                  0
cid                   0
stdv_paccfscore     330
cfcontrol          6016
thomasid              0
leg_totalbills     1369
feccandid             0
party              3327
state              3327
chamber            3327
amtsumlag_1           0
catcode               0
amtsumfwd_1           0
unemployment          0
billsum               0
loansum               0
amtsum                0
dtype: int64

In [317]:
year_bill_amount.drop_duplicates(inplace=True, keep='first')

In [318]:
year_bill_amount.isnull().sum()

date                  0
thomasid              0
years_exp           130
cid                   0
stdv_paccfscore     330
cfcontrol          6037
leg_totalbills     1371
feccandid             0
party              3338
state              3338
chamber            3338
amtsumlag_1           0
catcode               0
amtsumfwd_1           0
unemployment          0
billsum               0
loansum               0
amtsum                0
dtype: int64

In [319]:
year_bill_amount.to_csv('crsp.dime.mpl.df.K1', index=False)